In [1]:
from tensorflow import config

gpu_devices = config.experimental.list_physical_devices('GPU')

for device in gpu_devices: config.experimental.set_memory_growth(device, True)

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1 2 3"

# Jump here to start with prosessed csv file!!

In [3]:
import pandas as pd
import ast

train_data = pd.read_csv('./data/train_data_vector_complete.csv')
train_data['nutrient_vector'] = train_data['nutrient_vector'].apply(ast.literal_eval)
train_data['ingredient_vector'] = train_data['ingredient_vector'].apply(ast.literal_eval)

train_data['file_name'][3675] = 'sandwich129.png'
train_data['file_name'][1329] = 'cookies28.png'

train_data.head()

,title,image,ingredients,nutrients,file_name,parsed_ingredients,ingredient_vector,nutrient_vector
0,Vegan banana bread,https://images.immediate.co.uk/production/vola...,"['3 large black bananas', '75ml vegetable oil ...","{'calories': '218 calories', 'fatContent': '8 ...",bread1.jpg,"['black banana', 'sunflower oil', 'brown sugar...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[218.0, 8.0, 1.0, 33.0, 15.0, 2.0, 3.0, 0.5]"
1,Meatball & garlic bread traybake,https://images.immediate.co.uk/production/vola...,"['350g turkey thigh mince', '1 tsp dried orega...","{'calories': '565 calories', 'fatContent': '28...",bread2.jpg,"['thigh mince', 'tsp oregano', 'fennel seed', ...","[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[565.0, 28.0, 13.0, 34.0, 19.0, 6.0, 42.0, 1.3]"
2,Naan bread,https://images.immediate.co.uk/production/vola...,"['1x 7g sachet dried yeast', '2 tsp golden cas...","{'calories': '224 calories', 'fatContent': '8 ...",bread3.jpg,"['sachet yeast', 'caster sugar', 'bread flour'...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[224.0, 8.0, 4.0, 31.0, 3.0, 1.0, 6.0, 0.4]"
3,Pitta bread,https://images.immediate.co.uk/production/vola...,"['2 tsp fast-action dried yeast', '500g strong...","{'calories': '246 calories', 'fatContent': '2 ...",bread4.jpg,"['fastaction yeast', 'bread flour', 'tsp salt'...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[246.0, 2.0, 0.4, 47.0, 0.3, 2.0, 8.0, 1.0]"
4,Mincemeat banana bread,https://images.immediate.co.uk/production/vola...,"['150g unsalted butter, softened, plus extra f...","{'calories': '369 calories', 'fatContent': '23...",bread5.jpg,"['unsalted butter', 'caster sugar', 'egg', 'se...","[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[369.0, 23.0, 13.0, 37.0, 25.0, 2.0, 4.0, 0.5]"


# Build network

In [4]:
from keras.models import Model
from keras.layers import Dense, Input, Flatten, concatenate, GlobalAveragePooling2D
from keras.applications.densenet import DenseNet121

base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model.trainable = False

x = base_model.output

x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)

# use relu as activation to produce non-negative result
nutrient_pred = Dense(8, activation = "relu", name='nutrients')(x)
ingre_pred = Dense(100, activation = "sigmoid", name='ingredients')(x)

#out = concatenate([branchA, branchB])

model = Model(inputs=base_model.input, outputs=[nutrient_pred, ingre_pred])
model.summary()

Using TensorFlow backend.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [8]:
for l in model.layers:
    l.trainable = False
for l in model.layers[-4:]:
    l.trainable = True
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [9]:
from keras.preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(
    rescale=1/255, 
    validation_split=0.2, 
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    )

train_generator = datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='./data/train/',
    target_size = (256,256),
    x_col='file_name',
    y_col=['nutrient_vector', 'ingredient_vector'], # outputs for both tasks.
    batch_size=40,
    shuffle=True,
    subset='training',
    class_mode='multi_output')


valid_generator = datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='./data/train/',
    target_size = (256,256),
    x_col='file_name',
    y_col=['nutrient_vector', 'ingredient_vector'], # outputs for both tasks.
    batch_size=10,
    shuffle=True,
    subset='validation',
    class_mode='multi_output')

Found 3375 validated image filenames.
Found 843 validated image filenames.


In [ ]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

model.compile(optimizer=Adam(lr=1e-2, decay=0.99), loss={'nutrients':'mse', 'ingredients':'binary_crossentropy'}, 
              metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='./model.h5', verbose=1, 
                               monitor='val_nutrients_loss', mode='auto', save_best_only=True)

history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=round(train_generator.n/train_generator.batch_size)+1,
                    epochs=30,
                    validation_data=valid_generator,
                    validation_steps=round(valid_generator.n/valid_generator.batch_size),
                    callbacks=[checkpointer]
                   )

Epoch 1/30
39/85 [============>.................] - ETA: 40s - loss: 17115.1131 - nutrients_loss: 17114.7988 - ingredients_loss: 0.3116 - nutrients_accuracy: 0.9746 - ingredients_accuracy: 0.9280

/home/uceezku/project/lib64/python3.6/site-packages/PIL/Image.py:963: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


85/85 [==============================] - 100s 1s/step - loss: 12645.6061 - nutrients_loss: 12645.5352 - ingredients_loss: 0.2564 - nutrients_accuracy: 0.9881 - ingredients_accuracy: 0.9370 - val_loss: 6734.9849 - val_nutrients_loss: 8974.4600 - val_ingredients_loss: 0.2068 - val_nutrients_accuracy: 1.0000 - val_ingredients_accuracy: 0.9441

Epoch 00001: val_nutrients_loss improved from inf to 8974.45996, saving model to ./model.h5
Epoch 2/30
85/85 [==============================] - 97s 1s/step - loss: 6662.6796 - nutrients_loss: 6662.2964 - ingredients_loss: 0.1999 - nutrients_accuracy: 0.9997 - ingredients_accuracy: 0.9453 - val_loss: 5970.6660 - val_nutrients_loss: 7286.0947 - val_ingredients_loss: 0.2065 - val_nutrients_accuracy: 1.0000 - val_ingredients_accuracy: 0.9444

Epoch 00002: val_nutrients_loss improved from 8974.45996 to 7286.09473, saving model to ./model.h5
Epoch 3/30
85/85 [==============================] - 80s 939ms/step - loss: 5946.5628 - nutrients_loss: 5946.4009 - 

85/85 [==============================] - 66s 772ms/step - loss: 5544.6368 - nutrients_loss: 5544.2744 - ingredients_loss: 0.1971 - nutrients_accuracy: 0.9997 - ingredients_accuracy: 0.9455 - val_loss: 7745.4385 - val_nutrients_loss: 6252.5625 - val_ingredients_loss: 0.2017 - val_nutrients_accuracy: 1.0000 - val_ingredients_accuracy: 0.9451

Epoch 00020: val_nutrients_loss did not improve from 5754.88818
Epoch 21/30
85/85 [==============================] - 64s 749ms/step - loss: 5556.8637 - nutrients_loss: 5556.7715 - ingredients_loss: 0.1971 - nutrients_accuracy: 0.9997 - ingredients_accuracy: 0.9455 - val_loss: 6115.3916 - val_nutrients_loss: 5878.1694 - val_ingredients_loss: 0.2016 - val_nutrients_accuracy: 1.0000 - val_ingredients_accuracy: 0.9450

Epoch 00021: val_nutrients_loss did not improve from 5754.88818
Epoch 22/30
51/85 [=================>............] - ETA: 18s - loss: 5266.0624 - nutrients_loss: 5265.8638 - ingredients_loss: 0.1988 - nutrients_accuracy: 1.0000 - ingredie

In [67]:
model.save('./burger_first.h5')

# save model and history
#import pickle

#with open('./256256_100_8_20_datafrom.pkl','wb') as f:
    #pickle.dump(history, f)

#model.save_weights('./final_mse_unknown_4848_12000_subpixel_X4 _weights.hdf5')
#model.save('./model_and_history/perceptual_baseline1_3232_model.h5')

In [65]:
model.predict(np.expand_dims(normalized_imgs[0], axis=0))

[array([[375.68787  ,  33.96759  ,   1.8816065,  10.414915 ,  17.008928 ,
           3.2797508,  13.754762 ,   1.3811593]], dtype=float32),
 array([[1.23648286e-01, 1.00000000e+00, 0.00000000e+00, 8.03497434e-03,
         5.40461123e-01, 1.00000000e+00, 4.21894399e-26, 1.13232225e-01,
         1.00000000e+00, 1.90166916e-20]], dtype=float32)]

In [12]:
model.predict(np.expand_dims(normalized_imgs[0], axis=0))

[array([[4.4261505e+02, 2.4797510e+01, 8.7308254e+00, 3.2968742e+01,
         1.3308223e+01, 3.9988034e+00, 2.2814510e+01, 2.7198830e-01]],
       dtype=float32),
 array([[4.7885105e-01, 1.0450259e-01, 5.1347226e-02, 2.5539711e-01,
         3.4886116e-01, 3.6915123e-02, 2.6813054e-01, 1.6838061e-12,
         7.8342795e-02, 4.5565665e-03]], dtype=float32)]

In [66]:
print(nutrient_information[0])
print(ingre_vectors[0])

[737.0, 42.0, 10.0, 49.0, 6.0, 4.0, 39.0, 4.1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [56]:
model.predict(np.expand_dims(normalized_imgs[1], axis=0))

[array([[447.1117  ,  22.333237,   8.505314,  34.968132,   8.990719,
           4.908481,  18.69693 ,   1.106297]], dtype=float32),
 array([[0.07517844, 0.01071006, 0.5140303 , 0.07496288, 0.10428992,
         0.27532104, 0.30711216, 0.00156835, 0.01391488, 0.04083109]],
       dtype=float32)]

In [13]:
model.predict(np.expand_dims(normalized_imgs[1], axis=0))

[array([[417.40295  ,  18.817368 ,   8.588995 ,  37.723248 ,  11.250804 ,
           2.8133042,  17.200981 ,   5.309262 ]], dtype=float32),
 array([[6.0313046e-03, 7.9127848e-03, 5.5845326e-01, 3.8940400e-02,
         3.0152231e-02, 2.8474146e-01, 2.7315974e-02, 3.6718496e-11,
         4.6793133e-02, 3.7315071e-02]], dtype=float32)]

In [61]:
print(nutrient_information[1])
print(ingre_vectors[1])

[210.0, 5.0, 3.0, 35.0, 3.0, 2.0, 6.0, 0.4]
[0, 0, 0, 1, 1, 0, 0, 1, 1, 0]


In [58]:
model.predict(np.expand_dims(normalized_imgs[2], axis=0))

[array([[543.6993   ,  30.295036 ,  11.072008 ,  38.086296 ,  15.041439 ,
           2.8207948,  26.638327 ,   1.0462105]], dtype=float32),
 array([[0.77807677, 0.02893487, 0.0482401 , 0.7154445 , 0.02434561,
         0.04901221, 0.00301099, 0.03756481, 0.03010523, 0.2657691 ]],
       dtype=float32)]

In [62]:
print(nutrient_information[2])
print(ingre_vectors[2])

[654.0, 49.0, 13.0, 11.0, 4.0, 2.0, 40.0, 2.1]
[0, 1, 1, 0, 0, 1, 0, 0, 0, 1]
